In [ ]:
#Importing Libraries
import torch
from torch.autograd import Variable
import cv2
from data import BaseTransform, VOC_CLASSES as labelmap
from ssd import build_ssd
import imageio

In [ ]:
# Defining a Function that will do the Detections
def detect(frame, net, transform):
    height, width = frame.shape[:2] #here we're taking index 0 and 1, which correspond to the height and width dimensions of the frame shape
    #So now we have to undergo a few transformations for our original image (frame) into a torch Variable that will be accepted the SSD neural network
    #First we'll apply a transform transformation to make sure the frame has the right dimensions and right color values
    #Second, we'll convert this transform frame from a numpy array to a torch tensor. A torch tensor is more advanced matrix than a numpy array
    #Third, we'll add a fake dimension to the torch tensor, which will correspond to the batch. This is because the neural network cannot accept single inputs, such as single input vectors or a single input image. Instead, it only accepts them in batches of inputs
    #Last, we'll have to convert our torch tensor with the fake dimension into a Torch Variable, which contains both the tensor and a gradient. With this, we can feed the image into the neural network
    
    frame_t = transform(frame)[0] #Step 1, apply a transform to our frame. This transformation returns two elements, and we are only interested in taking the first element, index [0]. This is because only the first element, index[0], is the transformed frame with the right format (right colours and dimensions). frame_t is a numpy array
    x = torch.from_numpy(frame_t).permute(2,0,1) #Step 2, converting frame_t, which is a numpy array, into a torch tensor. Permute(2,0,1) made us go from RBG (0,1,2) to GRB (2,0,1). This now gives us the correct torch tensor format with the correct order of colors
    x = Variable(x.unsqueeze(0)) #Step 3, add a fake batch dimension to our torch tensor, x. The index of the dimension of the batch is always the first one, index 0. We create this batch dimension because the neural network cannot accept single inputs, such as single input vectors or a single input image. Instead, it only accepts them in batches of inputs. 
    #Step 4, which was converting our torch tensor with the fake batch dimension into a Torch Variable above. 
    
    y = net(x) #Inputting our torch variable, x, into our neural network, net, and output the prediction, y
    detections = y.data #so to get the prediction data from y, we just had to apply the .data to it. y as a whole contains other stuff and we only care about the prediction
    scale = torch.Tensor([width,height,width,height]) #So here we're creating a tensor of 4 dimensions. First two width,height correspond to the upper left corner of the rectangle detector, and the second width,height correspond to the lower right corner of the rectangle detector. We created this 4 dimension tensor so that we can normalize the scaler values of the positions (upper left and lower right corner positions) of the detected objects between 0 and 1. Tensors automatically scale between 0 and 1
    
    #The 4 elements of the detections tensor are [batch of outputs, number of classes, number of occurences of each class, tuple of 5 containing (score, x0, Y0, x1, Y1)]
    #so for the last element, the tuple with 5, the score is the score of whether it detected a class or not. The score will go from low to high for each occurence of each class. If the score for some occurences is less than 0.6, then the occurence of the class won't be found in the frame. x0,Y0,x1,Y1 correspond to the upperleft/lowerright corners of the rectangle detector, and these will only show if the score for the occurence is greater than 0.6
    for i in range(detections.size(1)):
        #index 1 is the number of classes, so the for loops goes from 0 to the number of classes
        j = 0 #j is the number of the occurence, currently being initalized as 0 
        while detections[0 ,i, j, 0] >= 0.6:
            #so we're saying while the score (index 0 in the tuple of 5) of class i and occurence j is greater than 0.6, keep the occurence
            pt = (detections[0, i, j, 1:] * scale).numpy() #so we're storing the x0,Y0,x1,Y1 dimensions of this occurence since score is greater than 0.6, so we can create a rectangle detector for it
            #we then applied the scale Tensor above so that we can do the normalization of the coordinates between 0 and 1. Then we converted the scaled tensor of 4 coordinates into a numpy array, so that we can use it in openCV to create the rectangle
            cv2.rectangle(img = frame, pt1 = (int(pt[0]),int(pt[1])), pt2 = (int(pt[2]), int(pt[3])), color = (255, 0, 0), thickness = 2 ) #creating the rectangle detector
            cv2.putText(img = frame, text = labelmap[i - 1], org = (int(pt[0]),int(pt[1])), fontFace = cv2.FONT_HERSHEY_SIMPLEX, fontScale = 3, color = (255,255,255), thickness = 3, lineType = cv2.LINE_AA) #so we're gonna put a text in the frame using labelmap, which will just correspond the detection number with a string representing the class, i. We did i - 1. since python starts at 0 but our labelmap starts at 1
            j += 1 #just incrementing j onto the next occurence
    return frame


In [ ]:
#Creating the SSD Neural Network
net = build_ssd('test') #we're just using a pre-trained model, so we're just gonna test the neural network rather than training one ourselves. 
net.load_state_dict(torch.load('ssd300_mAP_77.43_v2.pth', map_location = lambda storage, loc: storage)) #loading the pretrained weights
#please download the model here: https://drive.google.com/file/d/1yk85VYhQNFyy0Vgkw1pliJIUWV9mE6G4/view?usp=sharing 

In [ ]:
#Creating the Transformation
transform = BaseTransform(size = net.size, mean = (104/256.0, 117/256.0, 123/256.0)) #using the BaseTransform import to apply the transform. The first argument, size = net.size is the target size of the images to feed into the neural network
#The second argument, is a tuple to make sure the colour values are in the correct scale. This is just to meet convention
#we used this in Step 1 of the detect function, which was to make sure our frame was scaled with the correct size and colour

In [ ]:
#Performing Object Detection on the Video
reader = imageio.get_reader('your_video.mp4') #just using imageio to open the video
fps = reader.get_meta_data()['fps'] #getting the frames per second
writer = imageio.get_writer('your_video_output.mp4', fps = fps) #just using imageio to save a new video with the detections

for i, frame in enumerate(reader):
    #creating a for loop that'll go through all the frames in the video
    frame = detect(frame, net.eval(), transform)
    writer.append_data(frame) #creating the new video
    print(i) #just seeing the frame number in the video lmao
writer.close()